In [1]:
import arcpy
import time
import os

# Set start time
start_time = time.time()

# Set workspace environment
workspace = r"D:\median_barriers\median_barriers.gdb"  # Update to local GeoDataBase path
arcpy.env.workspace = workspace

# Set project folder location
project_folder = r"D:\median_barriers" # Update to project folder path

# Input data
roadway_data = r"D:\median_barriers\inputs\CA_HPMS_TMC2017\CA_HPMS_TMC2017\CA_HPMS_TMC2017.gdb\CA_HPMSPR2016_TMC2017" # Update to local path
median_barriers_csv = r"D:\median_barriers\inputs\initial_analysis_pairs.csv" # Update to local path
attribute_field = "ROUTE_ID"  # Select by attribute
attribute_value = "SHS_058_P"  # Update to selected highway in format "SHS_###_P"

# Output folder
output_folder = os.path.join(project_folder, "outputs") # Ensure that folder "outputs" exists within project folder

# Output data
dissolved_road = os.path.join(output_folder, "DissolvedRoad.shp")
median_points = os.path.join(output_folder, "MedianPoints.shp")
output_selected_roadway = os.path.join(output_folder, "SelectedRoadway.shp")
output_points_along_line = os.path.join(output_folder, "PointsAlongLine.shp")
output_points_to_line = os.path.join(output_folder, "PointsToLine.shp")
output_split_line = os.path.join(output_folder, "SplitLine.shp")
output_spatial_join = os.path.join(output_folder, "SpatialJoinResult.shp")
output_median_points_near = os.path.join(output_folder, "MedianPointsNear.shp")
output_table = os.path.join(output_folder, "SpatialJoinTable.xlsx")
output_spatial_join_cleaned = os.path.join(output_folder, "CleanedSpatialJoinResult.shp")

In [2]:
# Create new layer of selected roadway

# Select roadway data by attribute
arcpy.management.MakeFeatureLayer(roadway_data, "roadway_layer")
arcpy.management.SelectLayerByAttribute("roadway_layer", "NEW_SELECTION", f"{attribute_field} = '{attribute_value}'")
selected_roadway = arcpy.management.CopyFeatures("roadway_layer", output_selected_roadway)

In [3]:
# Dissolve selected roadway into single feature
dissolved_road = arcpy.management.Dissolve(selected_roadway, dissolved_road)

In [4]:
# Generate points along line at specified segment distance
arcpy.management.GeneratePointsAlongLines(dissolved_road, output_points_along_line, "DISTANCE", "100 Meters") # Distance between points is 100m

<Result 'D:\\median_barriers\\outputs\\PointsAlongLine.shp'>

In [5]:
# Convert points to line
arcpy.management.PointsToLine(output_points_along_line, output_points_to_line)

<Result 'D:\\median_barriers\\outputs\\PointsToLine.shp'>

In [6]:
# Split line by points to create selected roadway with segments of specified segment distance
arcpy.management.SplitLineAtPoint(output_points_to_line, output_points_along_line, output_split_line, "1 Meters") # Line searches for points within 1m

<Result 'D:\\median_barriers\\outputs\\SplitLine.shp'>

In [7]:
# Convert median points CSV to points layer
median_points = arcpy.management.MakeXYEventLayer(median_barriers_csv, "Longitude", "Latitude", "MedianPoints_Layer")

# Convert median points layer to feature class
median_points_fc = "MedianPoints_fc"
arcpy.management.CopyFeatures(median_points, median_points_fc)

<Result 'D:\\median_barriers\\median_barriers.gdb\\MedianPoints_fc'>

In [8]:
# Perform Near analysis to determine distance of median points from nearest roadway edge
near_table = arcpy.analysis.Near(median_points_fc, output_split_line, "100 Meters", "LOCATION", "NO_ANGLE", "PLANAR", "NEAR_FID NEAR_FID;NEAR_DIST NEAR_DIST;NEAR_X NEAR_X;NEAR_Y NEAR_Y") # Point searches for line within 100m

In [9]:
# Shift position of median points to intersect roadway line
arcpy.defense.CoordinateTableToPoint(median_points_fc, output_median_points_near, 'NEAR_X', 'DD_2', 'NEAR_Y', 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision') # CRS: WGS84

<Result 'D:\\median_barriers\\outputs\\MedianPointsNear.shp'>

In [10]:
# Spatial join median points to roadway line feature

# Perform Spatial Join
arcpy.analysis.SpatialJoin(output_split_line, output_median_points_near, output_spatial_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "Intersect", "1 Meters") # Line searches for points within 1m

<Result 'D:\\median_barriers\\outputs\\SpatialJoinResult.shp'>

In [11]:
# Export joined roadway table to Excel
arcpy.conversion.TableToExcel(output_spatial_join, output_table, "NAME", "CODE")

<Result 'D:\\median_barriers\\outputs\\SpatialJoinTable.xlsx'>

In [19]:
# Make a copy of the joined roadway line feature class and clean up duplicate fields (do not delete TARGET_FID)

# Copy the feature class
arcpy.CopyFeatures_management(output_spatial_join, output_spatial_join_cleaned)

# Set fields to delete
delete_fields = "Join_Count;Id;ORIG_FID;ORIG_SEQ;PAIR_NAME;VALID;ROUTE_ID;PAIR_ID;TRANSECT_I;PAIR_TYPE;Primary_Me;MEDIAN_WID;Secondary_;Latitude;Longitude;Notes;Initials;Field14;NEAR_FID;NEAR_DIST;NEAR_X;NEAR_Y;DDLat;DDLon;ORIG_OID"

# Delete fields
arcpy.management.DeleteField(output_spatial_join_cleaned, delete_fields, "DELETE_FIELDS")

<Result 'D:\\median_barriers\\outputs\\CleanedSpatialJoinResult.shp'>

In [13]:
# Calculate elapsed time
elapsed_time = time.time() - start_time
print(f"Completed successfully. Total elapsed time: {elapsed_time/60:.2f} minutes.")

Completed successfully. Total elapsed time: 1.72 minutes.


In [14]:
# This method does not automatically assign median attributes for roadway segments between two median points.
# For example, if I have a concrete/veg median pair, I will have three points with three median attributes: veg, transition, concrete.
# This method only assigns the median attributes of these points to roadway segments within the spatial join search distance (1m).
# This low search distance helps to ensure that only the NEAREST roadway segment is assigned the median attributes. This prevents overshoot.
# But, it also means the segments between the start/end points are not automatically assigned the median attributes.
# So, we need to manually update the segments between these points with their median attributes. 

# Next steps: fill in the missing median attributes

# 1) Open roadway Excel table in "outputs" folder
# 2) Sort the "ORIG_SEQ" column from smallest to largest
# 3) Select "Join_Count" column and Ctrl+F for '1'. Click Find Next to locate first median point
# 4) This row should have a valid median type (i.e. NOT "tran" or empty)
# 5) Copy the row cells (from column "PAIR_NAME" to "ORIG_OID") DOWN until you reach the next row with a "Join_Count"=1
# 6) This row should be the transition segment ("PAIR_TYPE" = "tran")
# 7) Record the number of rows for the transect you just copied in the Google Sheet under "Segment_Number"
# 8) After the transition segment, scroll down until you find the next row with a "Join_Count"=1
# 9) This row should have a valid median type (i.e. NOT "tran" or empty)
# 10) Copy the row cells (from column "PAIR_NAME" to "ORIG_OID") UP until you reach the transition segment
# 11) Record the number of rows for the transect you just copied in the Google Sheet under "Segment_Number"
# 12) Repeat until no rows with "Join_Count"=1 remain. Save the Excel file (DO NOT move it out of the "outputs" folder)
# 13) Now, we need to join these updated road segments with median attributes to the CROS roadkill data back in ArcGIS
# 14) Move onto the second script file
